# Preparing Libraries & Dependencies


In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3909, done.
remote: Counting objects: 100% (3909/3909), done.
remote: Compressing objects: 100% (3012/3012), done.
remote: Total 3909 (delta 1129), reused 2006 (delta 844), pack-reused 0
Receiving objects: 100% (3909/3909), 49.65 MiB | 18.48 MiB/s, done.
Resolving deltas: 100% (1129/1129), done.


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .

In [ ]:

# Modify setup.py file to install the tf-models-official repository targeted at TF v2.8.0
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.12.0', s)
    f.write(s)

In [ ]:
!pip install tensorflow
!pip install /content/models/research/
# !pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696985 sha256=1023581854afcb88672e0f0c7678ec7f1b9cebb3e409db4567bf6a7b3e754eb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-vsob6s5z/wheels/53/dd/70/2de274d6c443c69d367bd6a5606f95e5a6df61aacf1435ec0d
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [ ]:
import os
# import pathlib

import matplotlib
import matplotlib.pyplot as plt
from google.colab import drive
# import io
# import scipy.misc
import numpy as np
# from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
# from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub
import object_detection
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2

from google.protobuf import text_format



ModuleNotFoundError: ignored

!!**sA Introduci Copy for Apache License V2**

In [ ]:
# Run Model Bulider Test file, just to verify everything's working properly
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py


2023-06-07 06:40:45.418245: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
Running tests under Python 3.10.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0607 06:40:48.806312 140391081752384 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with para

# ***Chosing pretrained model***

In [ ]:
chosen_model = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
# chosen_model = 'ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'

tf_zoo = "http://download.tensorflow.org/models/object_detection/tf2/20200711/"
model_link = tf_zoo + chosen_model+ '.tar.gz'
model_archive = chosen_model+ '.tar.gz'

In [ ]:
if "pretrain_model" in os.listdir():
    !rm -rf pretrain_model
!mkdir pretrain_model
%cd /content/pretrain_model
!wget {model_link}
!tar -xvf {model_archive}

%cd /content

/content/pretrain_model
--2023-06-12 20:39:13--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.197.128, 2607:f8b0:400e:c03::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.197.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M   120MB/s    in 0.2s    

2023-06-12 20:39:13 (120 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnl

# ***Preparing dataset (images)***

In [ ]:
drive.mount('/content/drive')
!cp /content/drive/MyDrive/Thesis/images.zip /content

!unzip -q images.zip
!mv /content/images/tain /content/images/train

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
replace images/validate/pic77_jpg.rf.9bea3c93f1b9618cb20772ad79664067.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# ***Preparing configuration***

In [ ]:
pipeline = "/content/pretrain_model/" + chosen_model +"/pipeline.config"

label_map = "label_map.pbtxt"
checkpoint = "/content/pretrain_model/" + chosen_model + "/checkpoint/ckpt-0"

xmls_train =  "/content/images/train/"
xmls_validate = "/content/images/validate/"
images_train =  "/content/images/train/"
images_validate = "/content/images/validate/"

tf_record_train_path = "/content/train.tfrecord"
tf_record_validate_path = "/content/validate.tfrecord"

In [ ]:
labels_list = ['lego-car-mirror', 'lego-door-2','lego-door-4', 'lego-long-4', 'lego-short-2']

In [ ]:
with open(label_map, 'w') as f:
    for i, class_name in enumerate(labels_list, 1):  # Indexing from 1
        f.write("item {\n")
        f.write("\tid: {}\n".format(i))
        f.write("\tname: '{}'\n".format(class_name))
        f.write("}\n")
!cat {label_map}

In [ ]:
import xml.etree.ElementTree as ET
from object_detection.utils import dataset_util

def xmls_to_dict(xml_file, images_path):
    data = {
        'height': 0,
        'width': 0,
        'filename': '',
        'image': '',
        'format': '',
        'xmin': [],
        'xmax': [],
        'ymin': [],
        'ymax': [],
        'label_text': [],
        'label_id': []
    }
    tree = ET.parse(xml_file)
    root = tree.getroot()

    data['filename'] = root.find('filename').text
    data['width'] = int(root.find('size')[0].text)
    data['height'] = int(root.find('size')[1].text)
    data['format'] = b'jpg'

    for objectt in root.findall('object'):
        data['xmin'].append(int(objectt[5][0].text) / data['width'])
        data['xmax'].append(int(objectt[5][1].text) / data['width'])
        data['ymin'].append(int(objectt[5][2].text) / data['height'])
        data['ymax'].append(int(objectt[5][3].text) / data['height'])
        data['label_text'].append(objectt[0].text)
        data['label_id'].append(labels_list.index(objectt[0].text) + 1)

    with tf.io.gfile.GFile(images_path + data['filename'], 'rb') as t_image:
        data['image'] = t_image.read()


    return data

def generate_feature_dict(data_dict):
    tf_example={
        'image/filename': dataset_util.bytes_feature(data_dict['filename'].encode('utf8')), #has type str, but expected one of: bytes
        'image/height': dataset_util.int64_feature(data_dict['height']),
        'image/width': dataset_util.int64_feature(data_dict['width']),
        # 'image/source_id': dataset_util.bytes_feature(data_dict['filename']),
        'image/encoded': dataset_util.bytes_feature(data_dict['image']),
        'image/format': dataset_util.bytes_feature(data_dict['format']),
        'image/object/bbox/xmin': dataset_util.float_list_feature(data_dict['xmin']),
        'image/object/bbox/xmax': dataset_util.float_list_feature(data_dict['xmax']),
        'image/object/bbox/ymin': dataset_util.float_list_feature(data_dict['ymin']),
        'image/object/bbox/ymax': dataset_util.float_list_feature(data_dict['ymax']),
        'image/object/class/text': dataset_util.bytes_list_feature([l.encode('utf8') for l in data_dict['label_text']]),
        'image/object/class/label': dataset_util.int64_list_feature(data_dict['label_id'])
    }
    return tf_example


In [ ]:
import glob
tf_record = tf.io.TFRecordWriter(tf_record_train_path)
for xml_file in glob.glob(xmls_train + '/*.xml'):
    data_dict = xmls_to_dict(xml_file, images_train)
    feature_dict = generate_feature_dict(data_dict)
    tf_example = tf.train.Example(features=tf.train.Features(feature = feature_dict))
    tf_record.write(tf_example.SerializeToString())
tf_record.close()


In [ ]:
import glob
tf_record = tf.io.TFRecordWriter(tf_record_validate_path)
for xml_file in glob.glob(xmls_validate + '/*.xml'):
    data_dict = xmls_to_dict(xml_file, images_validate)
    feature_dict = generate_feature_dict(data_dict)
    tf_example = tf.train.Example(features=tf.train.Features(feature = feature_dict))
    tf_record.write(tf_example.SerializeToString())
tf_record.close()


In [ ]:
batch_size = 10
num_steps = 1000

In [ ]:
copy_pipeline = "/content/copypipeline.config"
!cp {pipeline} {copy_pipeline}
!cat {copy_pipeline}

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(copy_pipeline, "r") as pipe:
    proto_str = pipe.read()
    text_format.Merge(proto_str, pipeline_config)
pipeline_config.model.ssd.num_classes = len(labels_list)
pipeline_config.train_config.batch_size = batch_size
pipeline_config.train_config.fine_tune_checkpoint = checkpoint
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= label_map
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = tf_record_train_path
pipeline_config.eval_input_reader[0].label_map_path = label_map
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = tf_record_validate_path
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(copy_pipeline, "wb") as f:
    f.write(config_text)



In [ ]:
!cat {copy_pipeline}

# ***Fine-tune the model***

In [ ]:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/Thesis/training/train'

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={copy_pipeline} \
    --model_dir=/content/drive/MyDrive/Thesis/training/ \
    --alsologtostderr \
    --sample_1_of_n_eval_examples=1
    # --num_train_steps={num_steps} \


2023-06-07 06:42:05.598963: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
W0607 06:42:09.186012 140452882999104 cross_device_ops.py:1387] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0607 06:42:09.212143 140452882999104 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:local

# ***Convert & save the new model***

In [ ]:
# Make a directory to store the trained TFLite model
directory = '/content/drive/MyDrive/Thesis/trained_model'
!mkdir {directory}


#  Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = '/content/drive/MyDrive/Thesis/training/'

!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir /content/drive/MyDrive/Thesis/training/ \
    --output_directory {directory} \
    --pipeline_config_path {copy_pipeline}


mkdir: cannot create directory ‘/content/drive/MyDrive/Thesis/trained_model’: File exists
2023-06-07 06:42:17.451658: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# /content/drive/MyDrive/Thesis/trained_model/saved_model
converter = tf.lite.TFLiteConverter.from_saved_model(directory+'/saved_model')
tflite_model = converter.convert()

with open('/content/drive/MyDrive/Thesis/custom_model_lite/detect.tflite', 'wb') as f:
  f.write(tflite_model)